In [19]:
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [24]:
STOCKS_FILE = 'magazine_stocks_clean.csv'
DOLLAR_PRICE_FILE = 'dollar_prices_clean.csv'
TWEETS_FILE = 'magazine_tweets_clean.csv'

In [25]:
data_stocks = pd.read_csv(STOCKS_FILE, sep=',')
data_dollar = pd.read_csv(DOLLAR_PRICE_FILE, sep=',')
tweets = pd.read_csv(TWEETS_FILE, sep=',')

In [27]:
merged_data = data_stocks[data_stocks['date'] < "2019-10-19 00:00:00"]
merged_data.drop(merged_data.columns[[0]], axis=1, inplace=True)

In [28]:
merged_data['dollar_open'] = np.nan
merged_data['dollar_high'] = np.nan
merged_data['dollar_low'] = np.nan
merged_data['dollar_close'] = np.nan
merged_data.sort_values('date', inplace=True)

for i in merged_data.index:
    current_date = merged_data.at[i, 'date']
    current_day, current_time = current_date.split(' ')
    hour, min_sec = current_time.split(':', 1)

    if current_day == "2019-09-23":
        dollar_minuto_inform = data_dollar.loc[data_dollar['date'] == "2019-09-20 21:00:00"]
    else:
        dollar_minuto_inform = data_dollar.loc[data_dollar['date'] == current_day + ' ' + str(int(hour) + 4) + ':' + min_sec]

    merged_data.at[i, 'dollar_open'] = dollar_minuto_inform.iloc[0]['open']
    merged_data.at[i, 'dollar_high'] = dollar_minuto_inform.iloc[0]['high']
    merged_data.at[i, 'dollar_low'] = dollar_minuto_inform.iloc[0]['low']
    merged_data.at[i, 'dollar_close'] = dollar_minuto_inform.iloc[0]['close']

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [29]:
merged_data['sentiment_polarity'] = 0.0
merged_data['sentiment_subjectivity'] = 0.0
merged_data.head()

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\programdata\anaconda3\envs\stock-market\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,date,open,high,low,close,volume,dollar_open,dollar_high,dollar_low,dollar_close,sentiment_polarity,sentiment_subjectivity
0,1,2019-09-10 09:07:00,33.0013,33.0316,32.7213,32.8513,0,4.1004,4.1015,4.0974,4.1007,0.0,0.0
1,2,2019-09-10 09:08:00,32.8810,33.1225,32.8522,33.0411,301337,4.1014,4.1020,4.0971,4.0994,0.0,0.0
2,3,2019-09-10 09:09:00,33.0200,33.3100,33.0200,33.1300,177100,4.0987,4.0995,4.0961,4.0979,0.0,0.0
3,4,2019-09-10 09:10:00,33.1506,33.1761,32.9759,33.0006,179428,4.0972,4.0990,4.0956,4.0971,0.0,0.0
4,5,2019-09-10 09:11:00,33.0012,33.1010,32.9008,32.9112,204443,4.0981,4.0990,4.0956,4.0974,0.0,0.0


In [30]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

start_dt = date(2019,9,9)
end_dt = date(2019,10,18)

weekdays = [6,7]
valid_dates = []
for dt in daterange(start_dt, end_dt):
    if dt.isoweekday() not in weekdays:
        valid_dates.append(dt.strftime("%Y-%m-%d"))

for i in range(1, len(valid_dates)):
    tweets_of_day = tweets.loc[(tweets['date'] > "{:s} 23:59:59".format(valid_dates[i - 1])) &
                               (tweets['date'] <= "{:s} 23:59:59".format(valid_dates[i]))]
    
    merged_data.loc[(merged_data['date'] >= "{:s} 23:59:59".format(valid_dates[i - 1])) & (merged_data['date'] <= "{:s} 23:59:59".format(valid_dates[i])), 'sentiment_polarity'] = tweets_of_day['sentiment_polarity'].mean()
    merged_data.loc[(merged_data['date'] >= "{:s} 23:59:59".format(valid_dates[i - 1])) & (merged_data['date'] <= "{:s} 23:59:59".format(valid_dates[i])), 'sentiment_subjectivity'] = tweets_of_day['sentiment_subjectivity'].mean()

c:\programdata\anaconda3\envs\stock-market\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [31]:
merged_data.to_csv(STOCKS_FILE.split('_', 1)[0] + "_dataset.csv", sep=',', encoding='utf-8')
